# Support Vector Machines (SVM)

## Introduction

**Problem Statement**: Lenders need to assess the risk associated with lending money to an individual. By predicting whether someone qualifies for a personal loan, lenders can estimate the likelihood of repayment based on various factors such as credit history, income, employment status, debt-to-income ratio and many other factors. This helps them make informed decisions about whether to approve or deny a loan application. Hence we can think of this as a classification problem - whether to approve or deny the loan. Hence, we want to try and predict whether a person qualifies for personal loan or not based on various continuous and categorical variable from the given data. This is the problem we are trying to solve.

**ML Model**: As this is a classification problem, we can use ML models which are good at classifying tasks like decision trees, logistic regression, SVM etc. SVMs perform well even in high-dimensional spaces, which means they can handle datasets with a large number of features. Also, SVMs are considered to be less prone to overfitting. This is why w will be using SVMs for our use case.





**Dataset**

I have used the dataset provided on a Kaggle competition: https://www.kaggle.com/datasets/krantiswalke/bank-personal-loan-modelling/data. The file contains data on 5000 customers. The data include customer demographic information (age, income, etc.), the customer's relationship with the bank (mortgage, securities account, etc.), and the customer response to the last personal loan campaign (Personal Loan). As the datset has been taken from a Kaggle competition, I do not have the information which is not provided in the description of the competition. This case is about a bank (Thera Bank) whose management wants to explore ways of converting its liability customers to personal loan customers (while retaining them as depositors). A campaign that the bank ran last year for liability customers showed a healthy conversion rate of over 9% success. This has encouraged the retail marketing department to devise campaigns with better target marketing to increase the success ratio with minimal budget.

The Dataset contains the following attribute Information:
1. ID : Customer ID
2. Age : Customer's age in completed years
3. Experience : Years of professional experience
4. Income : Annual income of the customer
5. ZIP Code : Home Address ZIP code.
6. Family : Family size of the customer
7. CCAvg : Avg. spending on credit cards per month
8. Education : Education Level ( Undergrad, Graduate, Advanced/Professional)
9. Mortgage : Value of house mortgage if any
10. Personal Loan : Did this customer accept the personal loan offered in the last campaign?
11. Securities Account : Does the customer have a securities account with the bank?
12. CD Account : Does the customer have a certificate of deposit (CD) account with the bank?
13. Online : Does the customer use internet banking facilities?
14. Credit card : Does the customer use a credit card issued by the bank.

We will use the data to find out what type of customers are more likely to take a loan based on who took a loan the previous year - which is teh data we have available. Hence, our dependent variable (target variable or variable which we aim to predict) is "Personal Loan" and everything else is the independent variable. As I have taken this dataset from Kaggle, I will speculate very briefly as to why the dataset owners chose these specific datapoints.
1. Ability to take a loan depends on your financial position - your income, how much you need to spend to maintain a family, where you live, an existing debt obligations, purpose of your loan. It also helps to know your level of education as that can determine many factors like income, ability to repay etc. This covers why the bank recorded the age, experience, income, zip code, family size, average spending per month, mortgage and how educated you are.
2. A bank will require some sort of security to insure themselves against the loan given to the customer. A simple way to gauge this is to know how many deposits the customer has with the bank, how many securities account they have, does the customer regularly use credit cards (and do they pay it off as well).
This covers the other variables as well.


## Implementation

In [ ]:
#Import all the necessary libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn import metrics

In [ ]:
#Read the csv file
df = pd.read_csv('/content/sample_data/Bank_Personal_Loan_Modelling.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/sample_data/Bank_Personal_Loan_Modelling.csv'

In [ ]:
#Display the first few rows of the DataFrame, which can be helpful for quickly inspecting the data and understanding its structure.
df.head()

In [ ]:
#Returns a list of column names in the DataFrame df. We can inspect the names of the columns in the dataset.
df.columns

In [ ]:
#Get a concise summary of the DataFrame df to get the DataFrame's structure, including the number of non-null entries, the data types of each column, and memory usage.
df.info()

Whenever we need to use a dataset for a Machine Learning task, we need to inspect it for 'NaN" values so as to not encounter any runtime errors. The code df.isna().sum() is used to count the number of missing values (NaN or Not a Number) in each column of the DataFrame df. The result is a Series where the index corresponds to the column names, and the values represent the number of missing values in each respective column.

In [ ]:
df.isna().sum()

In [ ]:
#Let us inspect the dimensions of the DataFrame df. i.e. number of sampeles(rows) and features(columns).
df.shape

In [ ]:
#Let us create a list called X containing all the features which we will use for prediction. Each feature corresponds to a different attribite.
#X is also called the Feature Space. The dimension of X is 13.
#In the coming section, we will define Y = ['personal loan'] which is out target variable (prediction).
X = ['ID', 'Age', 'Experience', 'Income', 'ZIP Code', 'Family', 'CCAvg','Education', 'Mortgage','Securities Account','CD Account', 'Online', 'CreditCard']

In [ ]:
#While checking for anamolies, I found that the experience column contained negatieve values.
#All the other columns dont have miscued values, hence I have not included them here.
df['Experience'].unique()

NameError: name 'df' is not defined

In [ ]:
#As Experience cannot be negative, we can assume that it was due to some fault and correct them by taking absolute values.
df['Experience'] = df['Experience'].abs()
df['Experience'].unique()

It is a good practise to do an 'outlier analysis' to veridy how outliers of categorical variables affect the distribution of the dataset. After we analyse the outliers, we can choose to either modify them or to rid our dataset of these outliers so that we do not negatovely influence the model. We will be doing the second. By the following code, we create a list of all our categorical variables and then try to visualize the distribution of each categorical variable in your dataset using box plots. This will help us identify any potential outliers and gain insights into the spread of the data for each of the categorical feature.

In [ ]:
catogorical = ['Age','Experience', 'Income', 'Family']

fig,ax = plt.subplots(2,2,figsize=(5,5))
fig.suptitle('Boxplots of Categorical Variables', fontsize=10)
for i in range(len(catogorical)):
    x = i//2
    y = i%2
    sns.boxplot(data = df,x=catogorical[i],ax=ax[x,y])
plt.tight_layout()

Outlier Identification: In a boxplot, outliers are represented as individual points beyond the whiskers of the plot. Whiskers typically extend to a certain multiple of the interquartile range (in most cases, 1.5 times) from the first and third quartiles. Observations beyond these whiskers are considered potential outliers.

We observe that the income plot has outliers in it. As previously decided, we will discard these and then process our dataset further.

In [ ]:
#I wrote code to try different threshold values to find the least value for which we have no outliers.
#I found this to be 160. Let us remove all entries who have an income>160. Now we have no categorica outliers in our dataset.
#Let us visualise this.
income_outlier_idx = df[df['Income']>160].index
df = df.drop(income_outlier_idx)

plt.subplots(1, 1, figsize=(5, 5))
plt.title('Income without outliers', fontsize=10)
plt.xlabel('Income')
sns.boxplot(x=df['Income'])
plt.show()

Now we finally have a setup which allows us to train a model without outliers.

In [ ]:
#As previously mentioned, the features we use for training are stored in the X - the feature space.
#And the target variable 'personal loan' in y.
x = df[X]
y = df['Personal Loan']

the '.shape' command is used to get the dimensions of the subset of the DataFrame df containing only the columns specified in the list X. Specifically, it returns the number of samples(rows) and features(columns) in this subset. We will do this for the target variable as well to verify that we have not missed anything and that the information for the same number of samples is present.

In [ ]:
#shape of feature space
x.shape

In [ ]:
#shape of target variable
y.shape

From the above, we know that we now have 4665 samples with their labels and 13 features per sample.

While training a Machine Learning model, we split our 'Supervised' Dataset into a Training Dataset and a Testing Dataset. We use the training dataset to 'train' our model and the testing dataset to verify how accurate our trained model is at predicting on unknown data (The training data is unknown to the model as we have not used it to train the paramaters).

  1. X_train: This will be the training set of features (size = 80% of training set).
  2. X_test: This will be the testing set of features (size = 20% of testing set).
  3. y_train: This will be the corresponding training set of target values (size = 80% of training set).
  4. y_test: This will be the corresponding testing set of target values (size = 20% of testing set).

We can use the train_test_split function which randomly splits the data into training and testing sets. By default, 80% of the data will be used for training (X_train and y_train), and 20% will be used for testing (X_test and y_test), as specified by the test_size=0.2 argument. It is a standard practise to use an 80-20 split. The motivation behind this is loosely the Pareto principle.The Pareto principle is not a mathematically guaranteed property, but many observed phenomena follow the Pareto principle. As it is seen ubiquitous in nature, we try to emulate the same conjecture with our model. After all, we are trying to emulate 'intelligence'.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size = 0.2)

**Scaling the Data:** Data scaling is the process of transforming the values of the features of a dataset till they are within a specific range, e.g. 0 to 1 or -1 to 1. This is to ensure that no single feature 'dominates' the distance calculations in an algorithm, and can help to improve the performance of the algorithm.

Scaling your data in machine learning is important because many algorithms use the L-2 norm between two data points in their computations, which is sensitive to the scale of the variables. If one variable is on a much larger scale than another, that variable will dominate the distance calculation, and the algorithm will be affected by that variable more than the other irrespective of the importance of that variable in being a predictor. Scaling the data can help to balance the impact of all variables on the distance calculation and can help to improve the performance of the algorithm.

Source: https://medium.com/codex/why-scaling-your-data-is-important-1aff95ca97a2

The following code uses the StandardScaler object from scikit-learn to standardize the features in both the training and testing sets. Standardization (or Z-score normalization) is a common preprocessing step in machine learning that helps to scale features to have zero mean and unit variance. As mentioned above, Standardizing features is important because it helps to ensure that all features contribute equally to the learning process, and it can improve the performance of certain machine learning algorithms. Keep in mind, we need to scale features on both - the training AND testing sets.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Now we have the scaled data which we can use to train our model. We will do this using the the SVC fucntion which is pre-defined in Python. The usual process is:
1. Define the ML model.
2. Fit the model to our training set.
3. Predict values of our testing set using this model.
4. Calculate accuracy.
5. If not happy with the results, repeat the process above with different paramaters.

In [ ]:
#Define the fucntion and fit the training set to the model
SVM_model = SVC()
SVM_model.fit(X_train,y_train)

#Predict values of the testing set using our model
SVM_pred = SVM_model.predict(X_test)

Now that we have trained our model, let us test the 'accuracy' of the model. There are many parameters to test this. The following code evaluates the performance of the Support Vector Machine (SVM) model on both the training and testing sets, using the accuracy_score and classification_report functions from scikit-learn.

A typical Question: What is training accuracy?
Answer: When we split the data into training and testing sets, the model fits its paramters based on the training set. We do not want our model to overfit the training set else it will lose it's ability to predict new unseen data. Say your model has an accuracy of 90% on the training set and 80% on the testing set. This means that, on an average you can expect your model to perform with 80% accuracy on new data and 90% on non-new data. Verifying this is important to avoid overfitting. This is the concept of training accuracy.

## Analysis

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

#fit the model to the training set to check training accuracy
y_pred_train = SVM_model.predict(X_train)
#fit the model to the testing set to check training accuracy
y_pred_test = SVM_model.predict(X_test)

#comparing predicted labels with actual labels to get the accuracy metric
train_accuracy = accuracy_score(y_train, y_pred_train)
test_accuracy = accuracy_score(y_test, y_pred_test)

print("Train Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)

print("\nClassification Report for Test Set:")
print(classification_report(y_test, y_pred_test))

Let us now plot the confusion matrix for our model. As this was explained in detail in the previous assignment, we will not be going through its intricacies again. We will visually represent the performance of your SVM model in terms of true positive, true negative, false positive, and false negative predictions. It provides a visual summary of how well your model is classifying the data.

In [ ]:
#Define a heatmap to
sns.heatmap(confusion_matrix(y_test,SVM_pred),annot = True)
plt.xlabel('y_test')
plt.ylabel('y_predicted')
plt.title("Confusion matrix for the testing dataset")

We have trained our model and have relevant parameters to gauge how well the model has performed. Based on the output from our Support Vector Machine (SVM) model, here's an analysis of its performance:

1. Training Accuracy: The model has a training accuracy of approximately 98.20%. This is quite high and indicates that the model has learned the training dataset very well.
2. Test Accuracy: The test accuracy is about 97.21%. This is also high and quite close to the training accuracy, which is a good sign. It suggests that the model generalizes well to new, unseen data.
3. Precision: For class 0 (which is the negative class), the precision is 0.97, meaning that when it predicts the negative class, it is correct 97% of the time. For class 1 (the positive class), the precision is 0.94, which is also high.
4. Recall: The recall for class 0 is 1.00, indicating that the model identified all actual negative class instances correctly. However, for class 1, the recall is 0.68, suggesting that it missed about 32% of the positive class instances. One reason for this can be a class imbalance.
5. F1-Score: The F1-score for class 0 is 0.99, which is excellent. For class 1, it is 0.79, which is lower than class 0 but still a reasonably good score. The F1-score combines precision and recall into a single metric, which is particularly useful if there's a class imbalance.
6. Support: The support tells you how many instances of each class there are in the test set. There are 861 instances of class 0 and 72 instances of class 1. This shows our class imbalance, which could be why the recall for class 1 is notably lower than for class 0.
7. Macro Average: The macro average for precision, recall, and the F1-score is 0.96, 0.84, and 0.89, respectively. These are average scores that give equal weight to each class. The lower macro average recall indicates that one class (class 1) is bringing down the average significantly due to its lower recall.
8. Weighted Average: The weighted average takes the support into account and gives a better representation of the overall performance when there's class imbalance. The weighted averages are quite high across the board (0.97 for both precision and F1-score, and 0.97 for recall), indicating your model is performing well on the data overall.

Overall, our SVM model seems to be performing very well on both the training and test sets. The main area of concern is the lower recall for the positive class (class 1), which indicates that the model is failing to identify a significant proportion of positive instances. Given the imbalance in class support, we may want to look into techniques for handling class imbalance, such as oversampling the minority class, undersampling the majority class, or using class weights. Additionally, we might want to investigate why the model is missing these instances - it could be due to a lack of representative features for the positive class or other factors related to the data quality or feature engineering.

Let us also try to get a basic idea of the pros and cons associated with SVMs.

**Pros of SVM:**

1. Effective in High-Dimensional Spaces: SVMs are effective even in high-dimensional spaces, making them suitable for complex real-world problems.
2. Robust against Overfitting: SVMs are less prone to overfitting compared to some other models. This is because they try to maximize the margin between classes, which reduces the risk of fitting noise in the data.
3. Versatility with Kernels: SVMs can use different kernel functions (e.g., linear, polynomial, radial basis function) to adapt to different types of data. This allows for great flexibility in modeling complex relationships.
4. Capable of Handling Non-Linear Relationships: With the use of non-linear kernels, SVMs can effectively model non-linear relationships between features and target variables.

**Cons of SVM:**

1. Computationally Intensive: Training an SVM can be computationally expensive, especially for large datasets. This is particularly true when using non-linear kernels.
2. Inefficiency with Large Datasets: SVMs may become impractical for very large datasets due to their computational complexity.
3. Less Intuitive Model Interpretation: Unlike some other models (e.g., decision trees), SVMs don't provide as much insight into the relationships between features and the target variable. They work well as black-box models but may not offer as much interpretability.
4. Not Well-Suited for Noisy Data: SVMs are less effective when the dataset has a lot of noise or overlapping classes, as they are sensitive to misclassifications.